# House Price Index

## Introduction

This makes use of the following data sources: 
* https://www.gov.uk/government/statistical-data-sets/uk-house-price-index-data-downloads-may-2022
* https://geoportal.statistics.gov.uk/datasets/register-of-geographic-codes-december-2023-for-the-united-kingdom
* https://geoportal.statistics.gov.uk/datasets/ons-postcode-directory-november-2023

## Imports & Settings

In [13]:
import csv

from pprint import pprint

import polars as pl

from IPython.display import display, HTML
from ipywidgets import widgets
# display all columns
pl.Config(tbl_cols=None)

In [16]:
housing_type = widgets.Dropdown(
    options=["detached", "semi_detached", "terrace"],
    value="semi_detached",
    description='Housing Type:',
)
housing_type.style.description_width = "auto"
display(housing_type)


postcode = widgets.Text(
    value="",
    description='Postcode:',
)
postcode.style.description_width = "auto"
display(postcode)

Dropdown(description='Housing Type:', index=1, options=('detached', 'semi_detached', 'terrace'), style=Descrip…

In [7]:
print(housing_type.value)

detached


## HPI Data
Source: https://www.gov.uk/government/statistical-data-sets/uk-house-price-index-data-downloads-may-2022

In [13]:
hpi = pl.read_csv("../data/Average-prices-Property-Type-2023-06.csv")
hpi = hpi.rename({col: col.lower() for col in hpi.columns})
hpi = hpi.select([
    "date",
    "region_name", 
    "area_code",
    f"{housing_type.value}_average_price", 
    f"{housing_type.value}_index",
    f"{housing_type.value}_monthly_change", 
    f"{housing_type.value}_annual_change"
])
hpi

date,region_name,area_code,detached_average_price,detached_index,detached_monthly_change,detached_annual_change,semi_detached_average_price,semi_detached_index,semi_detached_monthly_change,semi_detached_annual_change,terraced_average_price,terraced_index,terraced_monthly_change,terraced_annual_change,flat_average_price,flat_index,flat_monthly_change,flat_annual_change
str,str,str,f64,f64,str,str,f64,f64,str,str,f64,f64,str,str,f64,f64,str,str
"""1995-01-01""","""England""","""E92000001""",86314.15895,28.257874,null,null,51533.22543,27.436474,null,null,41489.82431,25.279664,null,null,45218.54082,23.762969,null,null
"""1995-01-01""","""Wales""","""W92000004""",66539.58684,32.491063,null,null,41043.45436,31.399881,null,null,32506.88477,30.777231,null,null,34061.27288,34.448112,null,null
"""1995-01-01""","""Inner London""","""E13000001""",194483.5365,16.399257,null,null,121073.17,15.327414,null,null,87553.48096,14.627111,null,null,73707.69351,15.492239,null,null
"""1995-01-01""","""Outer London""","""E13000002""",160329.9602,22.303302,null,null,94802.27143,21.065017,null,null,70087.65516,20.040752,null,null,58266.86811,21.764751,null,null
"""1995-01-01""","""London""","""E12000007""",161449.3055,21.715622,null,null,95897.5293,20.321394,null,null,73705.96582,18.023197,null,null,64618.57236,17.858341,null,null
"""1995-01-01""","""East of Englan…","""E12000006""",88394.24421,25.892273,null,null,56278.555,24.527416,null,null,45356.64897,23.68537,null,null,40450.34445,26.116976,null,null
"""1995-01-01""","""North West""","""E12000002""",79786.62874,33.95164,null,null,46063.02541,32.234016,null,null,32974.56756,31.588852,null,null,34432.82565,34.028771,null,null
"""1995-01-01""","""South East""","""E12000008""",113412.4956,24.938444,null,null,66624.94926,23.874543,null,null,51088.69435,23.184027,null,null,43145.7917,25.379798,null,null
"""1995-01-01""","""West Midlands …","""E12000005""",81999.45317,31.385024,null,null,46457.18302,30.453267,null,null,36678.16759,29.405104,null,null,34983.15565,33.326469,null,null


## Reading Geography Data

Making changes to header records and quoting. 

In [38]:
def clean_csv(file: str, out: str) -> None:
    with open(file, 'r') as infile, open(out, 'w') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        for i, row in enumerate(reader):
            if i == 0:
                row = [x.lower().replace(" ", "_") for x in row]
            writer.writerow(row)

geog_files = [
    "../data/Register_of_Geographic_Codes_(December_2023)_UK.csv"
]

for file in geog_files:
    clean_csv(file, file.replace(".csv", "_clean.csv"))

Reading corrected data

In [44]:
postcode = "E"
geography = pl.read_csv("../data/geography_lookup.csv")
geography.filter(pl.col("related_entity_codes").str.contains(postcode))
# geography
        
            

﻿entity_code,entity_name,entity_abbreviation,entity_theme,entity_coverage,related_entity_codes,status,number_of_live_instances,number_of_archived_instances,number_of_cross-border_instances,date_of_last_instance_change,current_code_(first_in_range),current_code_(last_in_range),reserved_code_(for_chd_use),entity_owner_abbreviation,date_entity_introduced_on_rgc,entity_start_date,,_duplicated_0,_duplicated_1
str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str
"""J02""","""1961 Census Pa…","""PAR""","""Census""","""England and Wa…","""E92, W92""","""Current""","""11852""","""0""",0,"""n/a""","""J02000001""","""J02011852""","""J02999999""","""ONS""","""20/11/2018""","""20/11/2018""","""""","""""",""""""
"""J03""","""1961 Census Wa…","""WD""","""Census""","""England and Wa…","""E92, W92""","""Current""","""5212""","""0""",0,"""n/a""","""J03000001""","""J03005212""","""J03999999""","""ONS""","""20/11/2018""","""20/11/2018""","""""","""""",""""""
"""J04""","""1961 Census Di…","""LAD""","""Census""","""England and Wa…","""E92, W92""","""Current""","""1467""","""0""",0,"""n/a""","""J04000001""","""J04001467""","""J04999999""","""ONS""","""20/11/2018""","""20/11/2018""","""""","""""",""""""
"""J05""","""1961 Census Co…","""CTY""","""Census""","""England and Wa…","""E92, W92""","""Current""","""62""","""0""",0,"""n/a""","""J05000001""","""J05000062""","""J05999999""","""ONS""","""20/11/2018""","""20/11/2018""","""""","""""",""""""
"""K01""","""Travel to Work…","""TTWA""","""Other""","""United Kingdom…","""E30, W22, S22,…","""Current""","""6""","""12""",6,"""31/07/2015""","""K01000005""","""K01000014""","""K01999999""","""ONS""","""01/01/2009""","""01/01/2009""","""""","""""",""""""
"""K02""","""United Kingdom…","""UK""","""Administrative…","""United Kingdom…","""E92, W92, S92,…","""Current""","""1""","""0""",1,"""n/a""","""K02000001""","""K02000001""","""K02999999""","""ONS""","""31/10/2010""","""31/10/2010""","""""","""""",""""""
"""K03""","""Great Britain""","""GB""","""Administrative…","""Great Britain""","""E92, W92, S92""","""Current""","""1""","""0""",1,"""n/a""","""K03000001""","""K03000001""","""K03999999""","""ONS""","""31/10/2010""","""31/10/2010""","""""","""""",""""""
"""K04""","""England and Wa…","""E&W""","""Administrative…","""England and Wa…","""E92, W92""","""Current""","""1""","""0""",1,"""n/a""","""K04000001""","""K04000001""","""K04999999""","""ONS""","""31/10/2010""","""31/10/2010""","""""","""""",""""""
"""K05""","""Built Up Areas…","""BUA""","""Census""","""England and Wa…","""E34, W37""","""Archived""","""0""","""11""",11,"""30/11/2022""","""K05000001""","""K05000011""","""K05999999""","""ONS""","""27/06/2013""","""27/03/2011""","""""","""""",""""""
